## Installation on Google Colab

The transformers package is not installed by default on Google Colab. So let's install it with pip: 

In [1]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.5MB 25.9MB/s 
     |████████████████████████████████| 3.3MB 36.5MB/s 
     |████████████████████████████████| 901kB 32.8MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 


## Sentiment analysis in English

In this article, we will use the high-level pipeline interface, which makes it extremely easy to use pre-trained transformer models.

Basically, we just need to tell the pipeline what we want to do, and possibly to tell it which model to use for this task.

Here we're going to do sentiment analysis in English, so we select the `sentiment-analysis` task, and the default model: 

In [3]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

The pipeline is ready, and we can now use it: 

In [3]:
classifier(["this is a great tutorial, thank you", 
            "your content just sucks"])

[{'label': 'POSITIVE', 'score': 0.9998582601547241},
 {'label': 'NEGATIVE', 'score': 0.9971919059753418}]

We sent two sentences through the pipeline. The first one is predicted to be positive and the second one negative with very high confidence. 

Sounds good! 

Now let's see what happens if we send in french sentences: 

In [4]:
classifier(["Ton tuto est vraiment bien", 
            "il est complètement nul"])

[{'label': 'POSITIVE', 'score': 0.7650704979896545},
 {'label': 'POSITIVE', 'score': 0.8282670974731445}]

This time, the classification does not work... 

Indeed, the second sentence, which means "this tutorial is complete crap", is classified as positive. 

That's not a surprise: the default model for the sentiment analysis task has been trained on English text, so it does not understand French.

### Sentiment analysis in Dutch, German, French, Spanish and Italian

So what can you do if you want to work with text in another language, say French? 

You just need to search the hub for a [french classification model](https://huggingface.co/models?filter=fr&pipeline_tag=text-classification&sort=downloads). 

Several models are available, and I decided to select [nlptown/bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment). 

We can specify this model as the one to be used when we create our `sentiment-analysis` pipeline: 

In [5]:
multilang_classifier = pipeline("sentiment-analysis", 
                                model="nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
multilang_classifier(["Ton tuto est vraiment bien", 
                      "il est complètement nul"])

[{'label': '5 stars', 'score': 0.5787978172302246},
 {'label': '1 star', 'score': 0.9223358035087585}]

And it worked! The second sentence is properly classified as very negative. 

You might be wondering while the confidence for the first sentence is lower. I'm pretty sure that it's because this sentence scores high on '4 stars' as well. 

Now let's try with an actual review for a restaurant near my place: 

In [8]:
import pprint
sentence="Contente de pouvoir retourner au restaurant... Quelle déception... L accueil peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me proposer à la place... Une pizza pas assez cuite et pour finir une glace pleine de glaçons... Et au gout très fade... Je pensais que les serveuses seraient plus aimable à l idée de retrouver leur clientèle.. Dommage"
pprint.pprint(sentence)

('Contente de pouvoir retourner au restaurant... Quelle déception... L accueil '
 'peu chaleureux... Un plat du jour plus disponible à 12h45...rien à me '
 'proposer à la place... Une pizza pas assez cuite et pour finir une glace '
 'pleine de glaçons... Et au gout très fade... Je pensais que les serveuses '
 'seraient plus aimable à l idée de retrouver leur clientèle.. Dommage')


In [9]:
multilang_classifier([sentence])

[{'label': '2 stars', 'score': 0.5843755602836609}]

2 stars! on Google Review, this review has 1 star. Not a bad prediction. 

## Translation English-French

Let's try and do a bit of translation, from English to French. 

Again, we search the hub, and we end up with this pipeline: 


In [10]:
en_to_fr = pipeline("translation_en_to_fr", 
                    model="Helsinki-NLP/opus-mt-en-fr")

In [12]:
en_to_fr("your tutorial is really good")

[{'translation_text': 'votre tutoriel est vraiment bon'}]

This works well. Let's translate in the other direction. For this, we need to change the task and the model:

In [13]:
fr_to_en = pipeline("translation_fr_to_en", 
                    model="Helsinki-NLP/opus-mt-fr-en")

In [14]:
fr_to_en("ton tutoriel est super")

[{'translation_text': 'Your tutorial is great.'}]

Excellent translation! 

## Zero-shot classification in French

Nowadays, very large deep-learning models are trained on very large datasets collected from the internet. 

These models know a lot already, so they don't need to learn much more. 

Typically, it's possible to fine tune these models to a specific use case like text classification with a very small additional specific dataset. This is called few-shots learning. 

And sometimes, we can even do zero-shot learning: specific tasks can be performed without any specific training. This is what we're going to do now. 

We search the hub for a french zero-shot classification model, and we create this pipeline: 

In [15]:
classifier = pipeline("zero-shot-classification", 
                      model="BaptisteDoyen/camembert-base-xlni")

In the example below, I propose a sequence to classify into categories, and I also specify the categories.

It's important to note that the model **has not been trained with these categories**, you could change them at will! 

In [16]:
sequence = "Colin est en train d'écrire un article au sujet du traitement du langage naturel"
candidate_labels = ["science","politique","education", "news"]
classifier(sequence, candidate_labels)     

{'labels': ['science', 'news', 'education', 'politique'],
 'scores': [0.4613836407661438,
  0.20861364901065826,
  0.20573210716247559,
  0.12427058815956116],
 'sequence': "Colin est en train d'écrire un article au sujet du traitement du langage naturel"}

The predicted probabilities are actually quite sound! This sentence is indeed about science, news, and education. And not at all related to politics. 

But if we try this one: 

In [21]:
sequence = "Laurent Wauquiez reconduit à la tête de la région Rhône-Alpes-Auvergne à la suite du deuxième tour des élections."
candidate_labels = ["politique", "musique"]
classifier(sequence, candidate_labels)   

{'labels': ['politique', 'musique'],
 'scores': [0.6573010087013245, 0.34269899129867554],
 'sequence': 'Laurent Wauquiez reconduit à la tête de la région Rhône-Alpes-Auvergne à la suite du deuxième tour des élections.'}

It works! 

Feel free to try other sentences and other categories. You can also change the model if you wish to do zero-shot classification in English or in another language. 

## Summarization in French

Summarizing text is an interesting application of transformers. 

Here, we use a model trained on a dataset obtained by scraping [https://actu.orange.fr/](https://actu.orange.fr/), again found on the Hugging Face hub:



In [22]:
summarizer = pipeline("summarization", 
                       model="moussaKam/barthez-orangesum-title")

Let's use the first two paragraphs of an article about Covid-19 read in Le Monde:

In [26]:
import pprint
sentence = "La pandémie ne marque pas le pas. Le variant Delta poursuit son essor planétaire au grand dam de pays impatients de retrouver une vie normale. La pandémie a fait près de quatre millions de morts dans le monde depuis que le bureau de l’Organisation mondiale de la santé (OMS) en Chine a fait état de l’apparition de la maladie fin décembre 2019, selon un bilan établi par l’Agence France-Presse (AFP) à partir de sources officielles, lundi à 12 heures. Les Etats-Unis sont le pays le plus touché tant en nombre de morts (603 967) que de cas. Le Brésil, qui compte 513 474 morts, est suivi par l’Inde (396 730), le Mexique (232 564) et le Pérou (191 899), le pays qui déplore le plus de morts par rapport à sa population. Ces chiffres, qui reposent sur les bilans quotidiens des autorités nationales de santé, sont globalement sous-évalués. L’Organisation mondiale de la santé (OMS) estime que le bilan de la pandémie pourrait être deux à trois fois plus élevé que celui officiellement calculé."
pprint.pprint(sentence)

('La pandémie ne marque pas le pas. Le variant Delta poursuit son essor '
 'planétaire au grand dam de pays impatients de retrouver une vie normale. La '
 'pandémie a fait près de quatre millions de morts dans le monde depuis que le '
 'bureau de l’Organisation mondiale de la santé (OMS) en Chine a fait état de '
 'l’apparition de la maladie fin décembre 2019, selon un bilan établi par '
 'l’Agence France-Presse (AFP) à partir de sources officielles, lundi à 12 '
 'heures. Les Etats-Unis sont le pays le plus touché tant en nombre de morts '
 '(603 967) que de cas. Le Brésil, qui compte 513 474 morts, est suivi par '
 'l’Inde (396 730), le Mexique (232 564) et le Pérou (191 899), le pays qui '
 'déplore le plus de morts par rapport à sa population. Ces chiffres, qui '
 'reposent sur les bilans quotidiens des autorités nationales de santé, sont '
 'globalement sous-évalués. L’Organisation mondiale de la santé (OMS) estime '
 'que le bilan de la pandémie pourrait être deux à trois fois pl

In [30]:
summarizer(sentence, max_length=80)

[{'summary_text': 'Coronavirus : près de 4 millions de morts dans le monde'}]

Terse, but not bad! 

## Named entity recognition in French

Named entity recognition can serve as the basis of many interesting apps!  

For example, one could analyse financial reports looking for dates, prices, company names. 

Let's see how to do this. 

Here, we use a french equivalent of BERT, called CamemBERT, fine-tuned for NER:

In [4]:
ner = pipeline("token-classification", model="Jean-Baptiste/camembert-ner")

In [7]:
nes = ner("Colin est parti à Saint-André acheter de la mozzarella")
pprint.pprint(nes)

[{'end': 5,
  'entity': 'PER',
  'index': 1,
  'score': 0.94243556,
  'start': 0,
  'word': '▁Colin'},
 {'end': 23,
  'entity': 'LOC',
  'index': 5,
  'score': 0.99605554,
  'start': 17,
  'word': '▁Saint'},
 {'end': 24,
  'entity': 'LOC',
  'index': 6,
  'score': 0.9967083,
  'start': 23,
  'word': '-'},
 {'end': 29,
  'entity': 'LOC',
  'index': 7,
  'score': 0.99609375,
  'start': 24,
  'word': 'André'}]


We need to do a bit or post-processing to aggregate named entities with the same type. 

Here is a simple algorithm to do so (it can certainly be improved!)


In [10]:
cur = None
agg = []
for ne in nes: 
  entity=ne['entity']
  if entity != cur: 
    if cur is None: 
      cur = entity
    if agg: 
      print(cur, ner.tokenizer.convert_tokens_to_string(agg))
      agg = []
      cur = entity
  agg.append(ne['word'])
print(cur, ner.tokenizer.convert_tokens_to_string(agg))


PER Colin
LOC Saint-André


We found two named entities: 

* PERSON: Colin
* LOCATION: Saint-André